In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.getcwd()

'C:\\Users\\u287911\\projects\\shap'

In [3]:
df_raw = pd.read_excel("shap.xlsx")
df_raw.head()

,id,v1,v2,v3,v4,v5,v1_con,v2_con,v3_con,v4_con,v5_con,pred
0,C1,64,61,90,61,65,-8,-3,-4,3,10,1
1,C2,50,49,76,1,28,-9,-8,-10,4,1,0
2,C3,54,9,47,52,41,-6,-2,-7,1,5,0
3,C4,3,64,40,37,17,-5,-5,10,7,9,1
4,C5,96,36,32,63,2,-9,-8,-2,3,5,0


In [4]:
df = df_raw.copy()

In [5]:
x_vars  = ['v1','v2','v3','v4','v5']
x_vars_con  = [x + '_con' for x in x_vars]

#

In [6]:
df['top_contributers'] = np.where(df.pred == 1, 
                                  df[x_vars_con].apply(lambda x: x.nlargest(3).index.tolist(), axis=1),
                                  df[x_vars_con].apply(lambda x: x.nsmallest(3).index.tolist(), axis=1))

df.head()

,id,v1,v2,v3,v4,v5,v1_con,v2_con,v3_con,v4_con,v5_con,pred,top_contributers
0,C1,64,61,90,61,65,-8,-3,-4,3,10,1,"[v5_con, v4_con, v2_con]"
1,C2,50,49,76,1,28,-9,-8,-10,4,1,0,"[v3_con, v1_con, v2_con]"
2,C3,54,9,47,52,41,-6,-2,-7,1,5,0,"[v3_con, v1_con, v2_con]"
3,C4,3,64,40,37,17,-5,-5,10,7,9,1,"[v3_con, v5_con, v4_con]"
4,C5,96,36,32,63,2,-9,-8,-2,3,5,0,"[v1_con, v2_con, v3_con]"


In [7]:
df = df.explode('top_contributers').reset_index(drop = True)
df.head()

,id,v1,v2,v3,v4,v5,v1_con,v2_con,v3_con,v4_con,v5_con,pred,top_contributers
0,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v5_con
1,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v4_con
2,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v2_con
3,C2,50,49,76,1,28,-9,-8,-10,4,1,0,v3_con
4,C2,50,49,76,1,28,-9,-8,-10,4,1,0,v1_con


In [8]:
df.top_contributers = df.top_contributers.str.replace('_con','')
df.head()

,id,v1,v2,v3,v4,v5,v1_con,v2_con,v3_con,v4_con,v5_con,pred,top_contributers
0,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v5
1,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v4
2,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v2
3,C2,50,49,76,1,28,-9,-8,-10,4,1,0,v3
4,C2,50,49,76,1,28,-9,-8,-10,4,1,0,v1


In [9]:
df['variable_value'] = df.lookup(df.index, df.top_contributers)
df['explanations'] = df.apply(lambda x:'%s = %s' % (x.top_contributers,x.variable_value),axis=1)
df.head()


,id,v1,v2,v3,v4,v5,v1_con,v2_con,v3_con,v4_con,v5_con,pred,top_contributers,variable_value,explanations
0,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v5,65,v5 = 65
1,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v4,61,v4 = 61
2,C1,64,61,90,61,65,-8,-3,-4,3,10,1,v2,61,v2 = 61
3,C2,50,49,76,1,28,-9,-8,-10,4,1,0,v3,76,v3 = 76
4,C2,50,49,76,1,28,-9,-8,-10,4,1,0,v1,50,v1 = 50


In [10]:
df_explanations = df.groupby('id').agg(explanations = ('explanations', lambda x: ', '.join(x))).reset_index()
df_explanations.head()

,id,explanations
0,C1,"v5 = 65, v4 = 61, v2 = 61"
1,C10,"v4 = 44, v5 = 13, v3 = 1"
2,C2,"v3 = 76, v1 = 50, v2 = 49"
3,C3,"v3 = 47, v1 = 54, v2 = 9"
4,C4,"v3 = 40, v5 = 17, v4 = 37"


In [11]:
df_final =  df_raw.merge(df_explanations)
df_final

,id,v1,v2,v3,v4,v5,v1_con,v2_con,v3_con,v4_con,v5_con,pred,explanations
0,C1,64,61,90,61,65,-8,-3,-4,3,10,1,"v5 = 65, v4 = 61, v2 = 61"
1,C2,50,49,76,1,28,-9,-8,-10,4,1,0,"v3 = 76, v1 = 50, v2 = 49"
2,C3,54,9,47,52,41,-6,-2,-7,1,5,0,"v3 = 47, v1 = 54, v2 = 9"
3,C4,3,64,40,37,17,-5,-5,10,7,9,1,"v3 = 40, v5 = 17, v4 = 37"
4,C5,96,36,32,63,2,-9,-8,-2,3,5,0,"v1 = 96, v2 = 36, v3 = 32"
5,C6,88,61,39,64,47,-5,-6,6,2,8,1,"v5 = 47, v3 = 39, v4 = 64"
6,C7,41,15,27,7,31,-10,-1,-7,8,9,0,"v1 = 41, v3 = 27, v2 = 15"
7,C8,53,56,55,93,96,-8,-4,3,3,1,1,"v3 = 55, v4 = 93, v5 = 96"
8,C9,58,89,90,3,7,-2,-8,-6,6,1,0,"v2 = 89, v3 = 90, v1 = 58"
9,C10,33,70,1,44,13,-4,-5,0,10,7,1,"v4 = 44, v5 = 13, v3 = 1"
